# HW2: ETL

The .csv file from <a href="https://drive.google.com/file/d/1adkkH5xlRLj_y78KdITb1sRVuKbsZJc9/view" target="_blank">this google drive link</a> is needed before we begin the assignment. The dataset was downloaded from gapminder.org

## Instructions

For this assignment, you will Extract, Transform, and Load (ETL) the data in the link above into <a href="" target="_blank">the following Dimensional Model</a>

## Importing Libraries

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sodapy import Socrata
from google.cloud import bigquery
from google.oauth2 import service_account